In [1]:
import numpy as np
import pandas as pd
df = pd.DataFrame([['a',1],['a',1],['a',2],['a',3],['a',4],['b',5],['b',5],['b',6],['b',6],['b',6]], columns=['g', 'n'])
df

,g,n
0,a,1
1,a,1
2,a,2
3,a,3
4,a,4
5,b,5
6,b,5
7,b,6
8,b,6
9,b,6


In [56]:
# 그룹별 숫자(개수) 구할때
df.groupby(['g']).size()

# 그룹별 개수 구할때
df.groupby('g').agg({'n':'count'})

# 그룹별 평균 구할때
df.groupby(['g']).mean()

# 그룹별 중앙값 구할때
df.groupby(['g']).median()

# 그룹별 편차 구할때
df.groupby(['g']).std()

# 그룹별 인덱스 가져올때
df.groupby(['g']).groups.keys()

# 그룹 agg 한 뒤에 인덱스 가져올때
df.groupby('g').agg({'n':'mean'}).sort_values('n').index[0]

# 그룹별 특정 index만 가져올때 (그룹 중 특정 그룹만 추출)
df.groupby('g').get_group('a')

# 그룹별 unique 한 숫자 구할때
df.groupby('g').agg({'nunique'})

# 그룹별 가장 많은 그룹의 인덱스
df.groupby('g').n.count().idxmax()

# 그룹별 가장 많은 그룹의 값
df.groupby('g').n.count().max()

# 그룹별 숫자 구하고 dataframe 형식으로 나타낼때
pd.DataFrame(df.groupby(['g']).size().reset_index(drop=True), columns=['c'])

# 그룹별 컬럼 이름에 prefix 넣을때
df.groupby('g').agg({'n':'sum'}).add_prefix('_')



g
a    0.363173
b   -0.408248
Name: n, dtype: float64

In [57]:
from scipy.stats import skew, kurtosis

# 그룹별 첨도구하기
df.groupby('g').n.apply(lambda x: skew(x))

# 그룹별 왜도구하기
df.groupby('g').n.apply(lambda x: kurtosis(x))

g
a   -1.371972
b   -1.833333
Name: n, dtype: float64

In [54]:
# 그룹별 이전 데이터 가져올때
df['prev'] = df.groupby('g').n.shift(periods=1, fill_value=0)
df

# 그룹별 이후 데이터 가져올때
df['next'] = df.groupby('g').n.shift(periods=-1, fill_value=0)
df

# 그룹별 특정 컬럼의 평균을 뺀 값을 구할때
df.groupby('g').n.apply(lambda x: x - x.mean())

# 그룹별 두 컬럼 상관계수 구하기
temp = df.groupby('g').apply(lambda x: x[['prev', 'next']].corr())
abs(temp.prev[temp.prev!=1])


g      
a  next    0.138675
b  next    0.198618
Name: prev, dtype: float64

In [20]:
# 그룹별 coefficient of variation(변동계수) 구할때
# 변동계수 = 표준편차 / 평균 => (std/ mean)
# 그룹별 특정 칼럼의 평균과 편차
temp = df.groupby('g').agg({'n':['std','mean']})
temp.columns = temp.columns.droplevel(0)
temp['cv'] = temp['std'] / temp['mean']
temp

,std,mean,cv
g,,,
a,1.303840,2.2,0.592655
b,0.547723,5.6,0.097808


In [23]:
# 그룹별 함수 생성하여 처리
# 그룹별 quantile 처리
def q1(x):
  return x.quantile(0.25)

def q3(x):
  return x.quantile(0.75)

temp = df.groupby(['g']).agg({'n':[q1,q3]})
temp.columns = temp.columns.droplevel(0)
temp.reset_index()

,g,q1,q3
0,a,1,3
1,b,5,6


In [41]:
df

,g,n,prev,next
0,a,1,0,1
1,a,1,1,2
2,a,2,1,3
3,a,3,2,4
4,a,4,3,0
5,b,5,0,5
6,b,5,5,6
7,b,6,5,6
8,b,6,6,6
9,b,6,6,0


In [48]:
# 그룹별 중복 제거하고 특정값이 가장 큰 데이터만 남길때(중복데이터중 첫번째로 나오는 것만 추출)
df.groupby(['g']).apply(lambda x: x.loc[x.n.idxmax()]).reset_index(drop=True)

# 그룹별 중복 제거하고 각 칼럼별 가장 큰값만 남길때
df.groupby(['g']).apply(lambda x: x.max()).reset_index(drop=True)

# 그룹별 가장 값이 큰 2개 구할때
df.sort_values(by=['n'], ascending=False).groupby('g').head(2)

,g,n,prev,next
7,b,6,5,6
8,b,6,6,6
4,a,4,3,0
3,a,3,2,4


In [58]:
# 그룹별 랭크 구할떄 
# method => average: 평균순위 / min: 최소 순위 / max: 최대 순위 
# / first: 데이터상 먼저 나타나는 관측치 / dense: min과 비슷하지만 그룹간 순위가 1씩 증가
df['rank'] = df.groupby('g').n.rank(method='min', ascending=False)
df


,g,n,prev,next,rank
0,a,1,0,1,4.0
1,a,1,1,2,4.0
2,a,2,1,3,3.0
3,a,3,2,4,2.0
4,a,4,3,0,1.0
5,b,5,0,5,4.0
6,b,5,5,6,4.0
7,b,6,5,6,1.0
8,b,6,6,6,1.0
9,b,6,6,0,1.0
